In [24]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.exceptions import RefreshError
from openai import OpenAI
import re

In [25]:
app = Flask(__name__)

In [26]:
client = OpenAI(base_url="http://0.0.0.0:3000/v1", api_key="vllm")
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

In [27]:
class AI_AGENT:
    def parse_email(self, email_text, attendees_list):
        attendee_str = ", ".join(attendees_list)
        # This is the final, most precise prompt
        response = client.chat.completions.create(
            model=MODEL_PATH,
            temperature=0.0, # Set to 0 for maximum predictability
            messages=[{
                "role": "user",
                "content": f"""
                You are a precision AI assistant for parsing meeting requests.
                You will be given Email Content and a List of Attendees.
                - **Email Content**: "{email_text}"
                - **List of Attendees**: "{attendee_str}"

                You must extract the following four fields:
                1.  **participants**: Use the emails from the provided List of Attendees if the email text is generic (like "Hi team"). Otherwise, use the names from the email.
                2.  **time_constraints**: Extract the DAY or RELATIVE DATE part ONLY (e.g., "Wednesday", "tomorrow", "2 weeks from now"). DO NOT include the clock time in this field.
                3.  **meeting_duration**: Calculate the total duration in minutes as an INTEGER. For example, '2 hours' is 120.
                4.  **meeting_time**: Extract the specific CLOCK TIME ONLY (e.g., "10:00 A.M."). If no specific clock time is mentioned, this MUST be null.

                IMPORTANT: Your entire response must be ONLY a single, valid JSON object with these four keys.
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

    def judger(self, email_text, llm_output):
        # This judger uses the same improved logic
        llm_output_str = json.dumps(llm_output, indent=2)
        response = client.chat.completions.create(
            model=MODEL_PATH,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                You are a logical AI judge. Verify and correct the extracted JSON against the original email.
                - **Original Email**: "{email_text}"
                - **Initial JSON**: {llm_output_str}

                Apply these strict rules for correction:
                - **time_constraints**: Must be the DAY or RELATIVE DATE ONLY (e.g., "Wednesday").
                - **meeting_time**: Must be the CLOCK TIME ONLY (e.g., "10:00 A.M.") or null.
                - **meeting_duration**: Must be an INTEGER representing total minutes.
                - **participants**: Must be valid emails based on the context.

                FINAL INSTRUCTION: Your output MUST BE ONLY the final, verified JSON object, enclosed in triple backticks (```json ... ```).
                """
            }]
        )
        raw_response_content = response.choices[0].message.content
        match = re.search(r'```json\s*(\{.*?\})\s*```', raw_response_content, re.DOTALL)
        if match:
            return json.loads(match.group(1))
        else:
            try: return json.loads(raw_response_content)
            except json.JSONDecodeError: return llm_output

In [28]:

# Configuration: Mapping Attendees to their Tokens
ATTENDEE_TO_TOKEN_USER_MAPPING = {
    'userone.amd@gmail.com': 'userone.amd@gmail.com',
    'usertwo.amd@gmail.com': 'usertwo.amd@gmail.com',
    'userthree.amd@gmail.com': 'userthree.amd@gmail.com',
}


In [29]:
def retrive_calendar_events(user, start, end):
    token_path = "../Keys/" + user.split("@")[0] + ".token"
    try:
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)
        events_result = service.events().list(calendarId='primary', timeMin=start, timeMax=end, singleEvents=True, orderBy='startTime').execute()
        return events_result.get('items', [])
    except (RefreshError, FileNotFoundError): return []

# Master Scheduler Function (Heavily Upgraded to handle relative times)
def find_meeting_slot(attendees, time_constraint, duration_mins, requested_time_str):
    all_users_calendars = {}
    today = datetime.datetime.now(pytz.timezone('Asia/Kolkata'))
    target_date = None
    time_constraint_lower = time_constraint.lower()

    # --- NEW: Robust Date Parsing Logic ---
    weeks_match = re.search(r'(\d+)\s+weeks?', time_constraint_lower)
    days_match = re.search(r'(\d+)\s+days?', time_constraint_lower)
    
    if 'tomorrow' in time_constraint_lower:
        target_date = today + datetime.timedelta(days=1)
    elif weeks_match:
        num_weeks = int(weeks_match.group(1))
        target_date = today + datetime.timedelta(weeks=num_weeks)
    elif days_match:
        num_days = int(days_match.group(1))
        target_date = today + datetime.timedelta(days=num_days)
    else:
        # Fallback to day-of-the-week logic
        days_map = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6}
        target_day = next((day_index for day_name, day_index in days_map.items() if day_name in time_constraint_lower), -1)
        if target_day != -1:
            days_ahead = (target_day - today.weekday() + 7) % 7
            target_date = today + datetime.timedelta(days=days_ahead)
        else:
            # If all else fails, default to the next day
            target_date = today + datetime.timedelta(days=1)

    # Weekend Avoidance Logic
    if target_date.weekday() >= 5: # Saturday or Sunday
        target_date += datetime.timedelta(days=(7 - target_date.weekday()))

    day_start = target_date.replace(hour=9, minute=0, second=0, microsecond=0)
    day_end = target_date.replace(hour=17, minute=0, second=0, microsecond=0)

    # The rest of the function continues as before...
    all_busy_slots = []
    for email in attendees:
        token_user = ATTENDEE_TO_TOKEN_USER_MAPPING.get(email)
        if not token_user: continue
        user_events = retrive_calendar_events(token_user, day_start.isoformat(), day_end.isoformat())
        all_users_calendars[email] = user_events
        for event in user_events:
            start_str = event.get('start', {}).get('dateTime')
            end_str = event.get('end', {}).get('dateTime')
            if start_str and end_str:
                all_busy_slots.append({"start": datetime.datetime.fromisoformat(start_str), "end": datetime.datetime.fromisoformat(end_str)})
    
    # Merge slots and find a time (logic is the same as the most advanced version)
    # ... [The rest of the find_meeting_slot logic remains the same] ...
    all_busy_slots.sort(key=lambda x: x["start"])
    merged_busy_slots = []
    if all_busy_slots:
        merged_busy_slots = [all_busy_slots[0]]
        for current_slot in all_busy_slots[1:]:
            last_slot = merged_busy_slots[-1]
            if current_slot["start"] < last_slot["end"]:
                last_slot["end"] = max(last_slot["end"], current_slot["end"])
            else:
                merged_busy_slots.append(current_slot)
    
    meeting_duration = datetime.timedelta(minutes=duration_mins)
    search_start_time = day_start
    if requested_time_str:
        try:
            parsed_time = datetime.datetime.strptime(requested_time_str.upper().replace('.', ''), "%I:%M %p").time()
            requested_start = day_start.replace(hour=parsed_time.hour, minute=parsed_time.minute)
            requested_end = requested_start + meeting_duration
            is_free = True
            for slot in merged_busy_slots:
                if max(requested_start, slot["start"]) < min(requested_end, slot["end"]):
                    is_free = False
                    search_start_time = max(search_start_time, slot["end"])
                    break
            if is_free and requested_start >= day_start and requested_end <= day_end:
                return {"EventStart": requested_start.isoformat(), "EventEnd": requested_end.isoformat()}, all_users_calendars
        except ValueError: pass 
    
    free_slots = []
    last_busy_end = search_start_time
    for slot in merged_busy_slots:
        if slot["start"] > last_busy_end:
            free_slots.append({"start": last_busy_end, "end": slot["start"]})
        last_busy_end = max(last_busy_end, slot["end"])
    if day_end > last_busy_end:
        free_slots.append({"start": last_busy_end, "end": day_end})

    for slot in free_slots:
        if slot["end"] - slot["start"] >= meeting_duration:
            start = slot["start"]
            return {"EventStart": start.isoformat(), "EventEnd": (start + meeting_duration).isoformat()}, all_users_calendars
    return None, all_users_calendars

In [30]:
def your_meeting_assistant(input_json):
    agent = AI_AGENT()
    email_content = input_json.get("EmailContent", "")
    attendees_from_input = [att["email"] for att in input_json.get("Attendees", [])]
    organizer = input_json.get("From")
    if organizer and organizer not in attendees_from_input:
        attendees_from_input.append(organizer)

    initial_ai_output = agent.parse_email(email_content, attendees_from_input)
    final_ai_output = agent.judger(email_content, initial_ai_output)

    try:
        duration = int(final_ai_output.get('meeting_duration', 30))
    except (ValueError, TypeError):
        duration = 30
    
    time_constraint = final_ai_output.get('time_constraints', 'tomorrow')
    requested_time = final_ai_output.get('meeting_time')
    
    found_slot, all_calendars = find_meeting_slot(attendees_from_input, time_constraint, duration, requested_time)
    
    final_output = input_json.copy()
    if found_slot:
        start_time, end_time = found_slot.get("EventStart"), found_slot.get("EventEnd")
    else:
        start_time, end_time = None, None

    final_output.update({
        "EventStart": start_time,
        "EventEnd": end_time,
        "Duration_mins": duration,
        "MetaData": {"AI_Response": final_ai_output},
        "Attendees": [{"email": email, "events": all_calendars.get(email, [])} for email in attendees_from_input]
    })
    return final_output

# Flask Server
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    new_data = your_meeting_assistant(data)
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

Thread(target=run_flask, daemon=True).start()
print("✅ Final, robust server started on port 5000.")

✅ Final, robust server started on port 5000.
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
